In [ ]:
!pip install skillsnetwork
import skillsnetwork

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

In [ ]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

In [ ]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [ ]:
# get the data
await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0321EN-SkillsNetwork/concrete_data_week3.zip", overwrite=True)

  0%|          | 0/30036 [00:00<?, ?it/s]

Saved to '.'


In [ ]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [ ]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')
## Type your answer here
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical'
)

Found 10001 images belonging to 2 classes.
Found 5001 images belonging to 2 classes.


In [ ]:
model = Sequential()

In [ ]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [ ]:
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.layers

In [ ]:
model.layers[0].layers

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 4098 (16.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [ ]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

<ipython-input-19-172e67583a70>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(


Epoch 1/2
101/101 [==============================] - 66s 554ms/step - loss: 0.0465 - accuracy: 0.9868 - val_loss: 0.0138 - val_accuracy: 0.9954
Epoch 2/2
101/101 [==============================] - 51s 509ms/step - loss: 0.0078 - accuracy: 0.9983 - val_loss: 0.0088 - val_accuracy: 0.9980


In [ ]:
model.save('classifier_resnet_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# get the data
await skillsnetwork.prepare("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip", overwrite=True)

  0%|          | 0/80045 [00:00<?, ?it/s]

Saved to '.'


In [ ]:
model = load_model('classifier_resnet_model.h5')

In [ ]:

generator = ImageDataGenerator(preprocessing_function=preprocess_input)

training_generator = generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode="categorical",
)
validation_generator = generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode="categorical",
)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [ ]:
vgg16 = Sequential()

In [ ]:
vgg16.add(VGG16(
    include_top=False,
    pooling="avg",
    weights="imagenet",
    input_shape=(image_resize, image_resize, 3)))


In [ ]:
vgg16.add(Dense(num_classes, activation="softmax"))


In [ ]:
vgg16.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
steps_per_epoch_training = len(training_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

vgg16_story = vgg16.fit_generator(
    training_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

<ipython-input-29-c60872cda947>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  vgg16_story = vgg16.fit_generator(


Epoch 1/2
301/301 [==============================] - 388s 1s/step - loss: 0.6932 - accuracy: 0.5005 - val_loss: 0.6932 - val_accuracy: 0.4999
Epoch 2/2
301/301 [==============================] - 388s 1s/step - loss: 0.6932 - accuracy: 0.4977 - val_loss: 0.6931 - val_accuracy: 0.5001


In [ ]:
vgg16.save("vgg16_model.h5")

In [ ]:
testing_generator = generator.flow_from_directory(
    "concrete_data_week4/test", target_size=(image_resize, image_resize), shuffle=False,
)

Found 500 images belonging to 2 classes.


In [ ]:
resnet50_perf = model.evaluate_generator(testing_generator)
print("Loss: {}".format(round(resnet50_perf[0], 5)))
print("Accuracy: {}".format(round(resnet50_perf[1], 5)))

<ipython-input-32-383efb3f44ef>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  resnet50_perf = model.evaluate_generator(testing_generator)


Loss: 0.00576
Accuracy: 1.0


In [ ]:
vgg16_perf = vgg16.evaluate_generator(testing_generator)
print("Loss: {}".format(round(vgg16_perf[0], 5)))
print("Accuracy: {}".format(round(vgg16_perf[1], 5)))

<ipython-input-33-fd22f212167d>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  vgg16_perf = vgg16.evaluate_generator(testing_generator)


Loss: 0.69315
Accuracy: 0.5


In [ ]:
predictions_vgg16 = vgg16.predict_generator(testing_generator, steps=1)


def print_prediction(prediction):
    if prediction[0] > prediction[1]:
        print("Negative ({}% certainty)".format(round(prediction[0] * 100, 1)))
    elif prediction[1] > prediction[0]:
        print("Positive ({}% certainty)".format(round(prediction[1] * 100, 1)))

for i in range(5):
    print_prediction(predictions_vgg16[i])

<ipython-input-34-76325b9add4e>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions_vgg16 = vgg16.predict_generator(testing_generator, steps=1)


Negative (50.1% certainty)
Negative (50.1% certainty)
Negative (50.1% certainty)
Negative (50.1% certainty)
Negative (50.1% certainty)


In [ ]:
predictions_resnet50 = model.predict_generator(testing_generator, steps=1)
for i in range(5):
    print_prediction(predictions_resnet50[i])

<ipython-input-35-c616ad375838>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions_resnet50 = model.predict_generator(testing_generator, steps=1)


Negative (99.9% certainty)
Negative (99.2% certainty)
Negative (98.7% certainty)
Negative (99.6% certainty)
Negative (99.9% certainty)
